In [1]:
"""
Script python pour ouvrir les fichiers de traces de clavier

"""

import matplotlib.pyplot as plt
import numpy as np
import time
import os


def read_int(f):
    ba = bytearray(4)
    f.readinto(ba)
    prm = np.frombuffer(ba, dtype=np.int32)
    return prm[0]


def read_double(f):
    ba = bytearray(8)
    f.readinto(ba)
    prm = np.frombuffer(ba, dtype=np.double)
    return prm[0]


def read_double_tab(f, n):
    ba = bytearray(8 * n)
    nr = f.readinto(ba)
    if nr != len(ba):
        return []
    else:
        prm = np.frombuffer(ba, dtype=np.double)
        return prm


def get_pics_from_file(filename):
    # Lecture du fichier d'infos + pics detectes (post-processing KeyFinder)
    f_pic = open(filename, "rb")
    info = dict()
    info["nb_pics"] = read_int(f_pic)
    info["freq_sampling_khz"] = read_double(f_pic)
    info["freq_trame_hz"] = read_double(f_pic)
    info["freq_pic_khz"] = read_double(f_pic)
    info["norm_fact"] = read_double(f_pic)
    tab_pics = []
    pics = read_double_tab(f_pic, info["nb_pics"])
    nb_trames = 1
    while len(pics) > 0:
        nb_trames = nb_trames + 1
        tab_pics.append(pics)
        pics = read_double_tab(f_pic, info["nb_pics"])
    f_pic.close()
    return tab_pics, info


def get_dataset():
    """
    returns the dataset in a dict with key : name of the key
    value: list of spikes
    """
    dataset = {}
    os.chdir('../data/')

    for filename in os.listdir():
        list_of_spikes, info = get_pics_from_file(filename)

        # we get the name of the key by removing "pics_" at the beginning and ".bin" at the end
        key = filename[5:-4]

        dataset[key] = list_of_spikes

    return dataset

In [ ]:
if __name__ == "__main__":
    pics_nokey, info = get_pics_from_file("../data/pics_NOKEY.bin")
    pics_pad0, info = get_pics_from_file("../data/pics_0.bin")
    
    print(len(pics_pad0))

    ######### Pics ############
    # NO KEY
    plt.figure(1)
    plt.subplot(211)
    plt.plot(range(1, info["nb_pics"] + 1), pics_nokey[0], 'ko')
    plt.xlabel('numéro de pic')
    plt.ylabel('valeur du pic')
    plt.title('no key')
    plt.ylim(0, 1.5)
    plt.grid(b=True, which='both')
    # PAD-0
    plt.subplot(212)
    plt.plot(range(1, info["nb_pics"] + 1), pics_pad0[0], 'ko')
    plt.xlabel('numéro de pic')
    plt.ylabel('valeur du pic')
    plt.title('PAD-0')
    plt.ylim(0, 1.5)
    plt.grid(b=True, which='both')
    #
    plt.show()
    
    


In [4]:
from sklearn.linear_model import LinearRegression
import numpy as np

lr = LinearRegression()

dataset = get_dataset()
d_list = list(dataset)

X = []
Y = []
i = 0
for k in dataset:
    X += dataset[k]
    Y += [i] * len(dataset[k])
    i += 1

lr.fit(X, Y)



LinearRegression()

In [32]:
x_pred, info = get_pics_from_file("../tohack/pics_LOGINMDP.bin")
#x_pred, info = get_pics_from_file("../data/pics_0.bin")

i_NOKEY = d_list.index('NOKEY')

res = []

for trame_pred in x_pred:
    t = [np.array(trame_pred)]
    predict_i = lr.predict(t)[0]
    predict_i = int(predict_i)
    if predict_i != i_NOKEY:
        res.append(d_list[predict_i])

print(res)

# print(len(res))
# tot_0 = 0
# for i in range(len(res)):
#     if res[i] == '3':
#         tot_0 += 1
# print(tot_0)

['5', 'O', 'X', 'O', '3', '3', 'O', 'X', 'O', 'X', 'O', '5', 'V', 'H', '5', 'J', 'O', 'O', '5', '3', '5', 'X', 'P', 'O', 'O', 'H', 'X', 'X', 'H', 'O', 'O', 'A', 'X', 'H', 'Z', '3', 'O', '5', 'X', 'V', 'O', 'H', 'J', '5', 'X', 'CTRL', 'X', 'A', 'X', 'X', 'X', '3', 'Z', '3', '3', '3', 'V', 'X', 'Z', 'O', 'X', 'O', '5', '3', 'X', 'Z', '5', '3', 'O', 'Z', '5', '5', 'V', 'U', 'O', 'Z', 'O', '3', 'O', 'Z', 'V', '5', 'O', '3', 'V', 'J', 'Z', 'Z', 'H', 'O', 'V', 'V', 'X', 'V', 'Z', '5', 'Z', '5', '5', 'X', 'O', 'X', 'A', 'O', 'O', 'A', 'X', 'H', 'X', '3', 'J', 'H', '5', 'X', '3', 'X', 'Z', 'O', '3', 'X', 'X', 'Z', 'O', '3', '5', '5', '3', '3', '3', 'O', 'A', 'X', 'J', '3', 'O', '5', 'V', 'O', 'O', 'Z', 'X', 'X', 'A', 'W', 'O', 'X', 'O', 'O', 'X', 'A', 'A', 'P', 'O', 'H', 'W', 'H', 'Z', 'O', 'Z', 'X', '3', '5', '5', 'J', 'O', '3', '5', '3', 'H', 'O', 'X', 'O', 'O', 'X', 'X', 'A', 'X', '3', 'J', '3', 'X', '3', 'X', 'Z', 'Z', 'X', 'CTRL', 'X', '3', 'O', '5', 'X', '3', 'O', 'H', '5', '3', 'J', '8'

In [20]:
for i_k in range(len(res)):
    if res[i_k] == 'CTRL' and res[i_k + 2] == 'SUPPR':
        print('##########')
        print(res[i_k:i_k + 22])
        


##########
['CTRL', 'U', 'SUPPR', 'D', '8', 'CTRL', 'U', '2', '0', 'D', '0', 'V', 'V', 'N', 'P', 'D', '0', 'A', '7', '3', '2', 'Y']
##########
['CTRL', '8', 'SUPPR', 'D', 'N', 'Q', 'P', '0', 'P', 'D', 'R', 'P', '8', 'D', 'U', 'U', 'X', 'U', '0', 'J', '0', '0']


In [34]:


d = get_dataset()

In [35]:
print(len(d['1'][0]))

a = 0

for k in d:
    for i in range(len(d[k])):
        assert len(d[k][i]) == 17
        a += 1
        
print(a)

17
351612


# KNN

In [37]:
from sklearn.neighbors import KNeighborsClassifier

dataset = get_dataset()
d_list = list(dataset)

knn = KNeighborsClassifier(n_neighbors=len(dataset))

X = []
Y = []
i = 0
for k in dataset:
    X += dataset[k]
    Y += [i] * len(dataset[k])
    i += 1

knn.fit(X, Y)



KNeighborsClassifier(n_neighbors=42)

In [41]:
x_pred, info = get_pics_from_file("../tohack/pics_LOGINMDP.bin")
#x_pred, info = get_pics_from_file("../data/pics_0.bin")

i_NOKEY = d_list.index('NOKEY')

res = []

aa = 0
for trame_pred in x_pred:
    if aa % 100 == 0:
        print(aa)
    aa += 1
    t = [np.array(trame_pred)]
    predict_i = knn.predict(t)[0]
    predict_i = int(predict_i)
    if predict_i != i_NOKEY:
        res.append(d_list[predict_i])

print(res)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
['8', '8', '8', 'L', 'E', 'L', 'E', '6', 'F', '2', '2', 'Y', '2', 'U', 'Q', '0', 'Q', '4', 'E', 'O', '2', 'CTRL', 'D', 'CTRL', 'A', 'Q', 'E', 'ENTER', 'M', 'U', 'K', 'Q', 'O', 'F', 'CTRL', 'CTRL', '2', 'G', 'L', '7', 'ENTER', '1', '7', 'Z', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL',

In [42]:
for i_k in range(len(res)):
    if res[i_k] == 'CTRL' and res[i_k + 2] == 'SUPPR':
        print('##########')
        print(res[i_k:i_k + 22])
        

##########
['CTRL', '0', 'SUPPR', 'SUPPR', 'SUPPR', '0', '0', '0', '0', '0', '0', '0', 'SUPPR', '0', 'ENTER', 'SUPPR', 'SUPPR', 'SUPPR', '0', 'SUPPR', 'SUPPR', 'SUPPR']
##########
['CTRL', '0', 'SUPPR', 'SUPPR', 'SUPPR', 'SUPPR', '0', 'SUPPR', 'CTRL', 'SUPPR', 'SUPPR', '0', '0', '0', 'M', '0', 'SUPPR', 'SUPPR', 'SUPPR', 'SUPPR', 'SUPPR', 'SUPPR']
##########
['CTRL', 'SUPPR', 'SUPPR', '0', '0', '0', 'M', '0', 'SUPPR', 'SUPPR', 'SUPPR', 'SUPPR', 'SUPPR', 'SUPPR', 'R', '0', 'SUPPR', 'SUPPR', '0', 'SUPPR', 'SUPPR', '0']
##########
['CTRL', '0', 'SUPPR', '0', 'SUPPR', 'SUPPR', 'SUPPR', '0', 'SUPPR', 'SUPPR', 'SUPPR', 'SUPPR', 'SUPPR', 'SUPPR', '0', 'SUPPR', '0', 'SUPPR', '0', 'SUPPR', 'SUPPR', 'SUPPR']


In [43]:
print(len(res))

4626
